# Prepare Data Pt. 2 (`data_prep_kriging`)

This emulates `data_prep_kriging.m`.  **IT MUST BE RUN AFTER `Prepare Locals.ipynb`, SINCE THAT CREATES `pkdata.hdf5`.**

In [1]:
import pandas as pd
import numpy as np

In [4]:
# Read in directions file.
# directions = pd.read_csv(('/mnt/c/Users/czhu5/Documents/VolumeModel/'
#                           'TEPS-dev/PRTCS/direction3.csv'))
# directions.sort_values('centreline_id', inplace=True)
# directions.reset_index(drop=True, inplace=True)
# pkstore = pd.HDFStore('./data/pkdata.hdf5', 'r+')
# pkstore.put('directions', directions)
# pkstore.close()

with pd.HDFStore('/home/czhu/Data/btp_sandbox_prep_kriglocaldata/pkdata2010.hdf5') as pkstore:
    directions = pkstore['directions']
    aadt_landuse_2_2010 = pkstore['aadt_landuse_2_2010']

In [5]:
directions.head()

,centreline_id,dir_bin
0,108,-1
1,108,1
2,117,-1
3,117,1
4,118,-1


In [6]:
aadt_landuse_2_2010.head()

,AADT,Sum_pop,number of lanes,speed limit,employment,commercial,industrial,government,road type
centreline_id,,,,,,,,,
103,NaN,7824.516305,0,0,1.025477e+06,29105.308526,1.122606e+06,33081.128020,NaN
106,NaN,8490.111947,0,0,2.746630e+05,84597.684531,3.996810e+05,322477.021673,NaN
107,NaN,7520.466102,0,0,2.146579e+05,84955.419497,3.293675e+05,352416.655710,NaN
108,NaN,7297.781592,2,45,2.507206e+05,84597.684531,3.589855e+05,301465.932090,37
112,NaN,8050.696644,0,0,1.923570e+05,99521.912333,2.920311e+05,418089.930909,NaN


In [7]:
aadt_landuse_2_2010.index.shape

(64848,)

### Produce `resmat{YEAR}.txt`

Equivalent to `b` in `data_prep_kriging.m`.

The exact definition of `b`'s colums is given by `b2`'s:
```
b2=[ref_id_start_point_t ref_id_stop_point count_t speed_limit_t population_t ...
    lane_num_t end_point_t start_point_t speed_limit_t.*lane_num_t commer_t ...
    indus_t gov_t emplo_t dummy_road_type_t Dist_t];
```

The only thing my `resmat` doesn't have is `speed_limit_t.*lane_num_t`, which is trivial to calculate.  (Also we have speed limits from two different sources, but I'm lazy.)

In [37]:
# Read in raw centreline-centreline distance table.
with pd.HDFStore('/home/czhu/Data/btp_sandbox_prep_kriglocaldata/resmat.hdf5') as rsm:
    resmat = rsm['resmat']
    resmat = resmat[['origin_centreline_id', 'dest_centreline_id',
                     'network_distance', 'speed_limit']]

In [38]:
resmat.shape

(4657659, 4)

In [39]:
resmat = resmat.join(aadt_landuse_2_2010, on='dest_centreline_id', how='left', rsuffix='_r')
resmat.shape

(4657659, 13)

In [40]:
# Auto-eliminates nulls.
resmat = resmat.loc[(resmat['number of lanes'] > 0) & (resmat['network_distance'] < 2.), :]

In [41]:
resmat.shape

(3011569, 13)

### Produce `distance_short{YEAR}.csv`

`ACSPedit{YEAR}.csv` is this but for reference IDs, and is unused.

In [42]:
distance_short = resmat[['origin_centreline_id', 'dest_centreline_id', 'network_distance']]

In [79]:
with pd.HDFStore('/home/czhu/Data/btp_sandbox_prep_kriglocaldata/distance_short.hdf5', 'w') as dsh:
    dsh['distance_short'] = distance_short

### Produce `data_for_pred{YEAR}.txt` and `data_for_fit{YEAR}.txt`

Their `id` counterparts are just the endpoint centreline_id versions of these.

In [64]:
# Equivalent to picking out the first unique instance of b(:7) in [~,idx]=unique(c,'rows'); out2=b(idx,:);
# Use speed limit from land use xlsx.
data_for_pred = resmat[['dest_centreline_id', 'speed limit', 'AADT', 'Sum_pop',
                        'number of lanes', 'employment', 'commercial',
                        'industrial', 'government', 'road type']]

In [65]:
data_for_pred = data_for_pred.drop_duplicates().reset_index(drop=True)

In [66]:
data_for_pred.head()

,dest_centreline_id,speed limit,AADT,Sum_pop,number of lanes,employment,commercial,industrial,government,road type
0,9109255,45.0,763.356954,12558.251379,2.0,1.084553e+06,NaN,1.312336e+06,187747.021115,37
1,14188125,0.0,NaN,8374.539150,2.0,7.119475e+05,NaN,4.601152e+05,359721.633864,NaN
2,914742,45.0,571.524269,10633.715781,2.0,6.337984e+05,NaN,7.766608e+05,308111.966045,37
3,914856,55.0,NaN,7071.569392,4.0,1.958703e+05,NaN,4.326051e+05,NaN,10
4,7585679,55.0,NaN,12267.637772,4.0,8.506309e+05,NaN,1.089469e+06,55399.259756,10


In [67]:
data_for_pred.shape

(18077, 10)

In [68]:
data_for_pred = data_for_pred.loc[data_for_pred['speed limit'] > 0., :]

In [69]:
ctrline_negdir = directions[directions['dir_bin'] < 0]

In [70]:
assert ctrline_negdir['centreline_id'].unique().shape[0] == ctrline_negdir.shape[0]

In [71]:
data_for_pred = (pd.merge(data_for_pred, ctrline_negdir[['centreline_id']],
                         left_on='dest_centreline_id', right_on='centreline_id')
                 .drop(columns='centreline_id').reset_index(drop=True))

In [85]:
data_for_pred = data_for_pred[data_for_pred['road type'] != '0'].reset_index(drop=True)

In [86]:
# The fitting data is any road section whose AADT estimate is 2000 or greater (automatically eliminates nulls).
data_for_fit = data_for_pred[data_for_pred['AADT'] > 2000.].reset_index(drop=True)

In [87]:
data_for_pred.head()

,dest_centreline_id,speed limit,AADT,Sum_pop,number of lanes,employment,commercial,industrial,government,road type
0,9109255,45.0,763.356954,12558.251379,2.0,1.084553e+06,NaN,1.312336e+06,187747.021115,37
1,914742,45.0,571.524269,10633.715781,2.0,6.337984e+05,NaN,7.766608e+05,308111.966045,37
2,914856,55.0,NaN,7071.569392,4.0,1.958703e+05,NaN,4.326051e+05,NaN,10
3,7585679,55.0,NaN,12267.637772,4.0,8.506309e+05,NaN,1.089469e+06,55399.259756,10
4,7586931,55.0,NaN,12233.734943,4.0,9.007790e+05,NaN,1.098013e+06,92275.284926,10


In [88]:
data_for_fit.head()

,dest_centreline_id,speed limit,AADT,Sum_pop,number of lanes,employment,commercial,industrial,government,road type
0,914446,55.0,8961.192262,9590.566862,4.0,1.871610e+06,NaN,1.366294e+06,352847.104820,10
1,914320,55.0,10756.432090,7671.348647,4.0,1.980414e+06,NaN,1.389806e+06,216803.398979,10
2,3326929,55.0,12739.531960,7602.013661,4.0,6.661395e+05,NaN,7.864940e+05,106614.317019,10
3,11070075,50.0,8240.331834,7394.967113,4.0,6.906023e+05,NaN,8.668984e+05,106614.317019,33
4,20048292,55.0,5824.497184,9216.707898,4.0,1.264567e+06,NaN,8.836344e+05,362648.479967,10


In [89]:
# Read in raw centreline-centreline distance table.
with pd.HDFStore('/home/czhu/Data/btp_sandbox_prep_kriglocaldata/data_for_pf2010negative.hdf5', 'w') as dfp:
    dfp['data_for_pred'] = data_for_pred
    dfp['data_for_fit'] = data_for_fit

## Check that `data_for_pred` contains major arterials

In [106]:
import pathlib, os
import configparser
import psycopg2
import geopandas as gpd

filepath = pathlib.Path.home().joinpath('.charlesconfig')
config = configparser.RawConfigParser()
config.read(filepath.as_posix());

with psycopg2.connect(database='bigdata', user=config['POSTGRES']['user'],
                      password=config['POSTGRES']['password'],
                      host=config['POSTGRES']['host'], port=config['POSTGRES']['port']) as db_con:
    gis_centreline = gpd.read_postgis('SELECT * FROM gis.centreline', db_con)
    gis_centreline['geo_id'] = gis_centreline['geo_id'].astype(int)

In [114]:
road_descriptions = pd.merge(data_for_pred, gis_centreline[['geo_id', 'fcode', 'fcode_desc']],
                             how='inner', left_on='dest_centreline_id', right_on='geo_id')

In [119]:
road_descriptions['fcode_desc'].value_counts()

Collector              5932
Major Arterial         5422
Minor Arterial         3269
Expressway              718
Expressway Ramp         584
Major Arterial Ramp     109
Local                   104
Collector Ramp           10
Name: fcode_desc, dtype: int64

In [120]:
road_descriptions_fit = pd.merge(data_for_fit, gis_centreline[['geo_id', 'fcode', 'fcode_desc']],
                                 how='inner', left_on='dest_centreline_id', right_on='geo_id')

In [121]:
road_descriptions_fit['fcode_desc'].value_counts()

Major Arterial         1797
Minor Arterial          901
Collector               730
Expressway              114
Expressway Ramp          43
Major Arterial Ramp      14
Local                     5
Name: fcode_desc, dtype: int64

Close enough, I guess.

In [125]:
road_descriptions = pd.merge(data_for_pred, gis_centreline[['geo_id', 'fcode', 'fcode_desc']],
                             how='left', left_on='dest_centreline_id', right_on='geo_id')

In [127]:
road_descriptions.loc[road_descriptions['geo_id'].isnull(), 'dest_centreline_id']

35         914545
307       3086207
416       7668066
770      14204001
1150      8407393
1291       909410
1373      9655693
1876      7963393
2253         8436
2306      8791748
2312     10759021
2355      8166335
2356      8929258
2457     14004162
2526      9781235
2620     11461945
2624       446598
3031      8206114
3032      8206105
3047     13974047
3182      6913362
3258      1140382
3310     14003783
3420      1139499
3466      1140633
3558      2981086
3559      1141672
3825     14673528
3891      4153107
3920     20235606
           ...   
14824     6257558
14885     6247978
14889     6247982
14890     6247983
14969     9468212
15108      106132
15169     4786680
15229      107510
15241      107539
15263    20040264
15285      106675
15292      105619
15294     6793391
15330     7195953
15360      105988
15369    14191397
15415      443550
15457    14258081
15480    13502910
15563      109757
15637     6674323
15679      105368
15880    14661315
15891    20057375
15893    1

So there are some roads whose codes don't exist in the current centreline.